## NBA Players
1. **Title** - Visualizing NBA Players on Latent Space

2. **Features**
 - Player_Info
     - DISPLAY_FIRST_LAST
     - BIRTHDATE
     - HEIGHT
     - WEIGHT
     - SCHOOL
     - POSITION
     - JERSEY
 - Career
     - Teams Played
     - Points Per Game
     - Assists Per Game
     - 3 Points Per Game
     - Steals Per Game
     - NBA Titles
     - MVP Titles
 
3. **Ideas**
 - Categorical Embeddings
 - Graphs
 - UMAP for dim reduction
 - Altair interactive for visualization

In [ ]:
#!pip install -U pandas

In [ ]:
from tqdm import tqdm_notebook
from nba_api.stats.endpoints import commonplayerinfo, playercareerstats
from nba_api.stats.static import players
from nba_api.stats.static import teams
import pandas as pd
import numpy as np
from datetime import datetime
pd.__version__

In [ ]:
# Find players by full name.
players.find_players_by_full_name('michael jordan')

In [ ]:
# Find players by full name.
players.find_players_by_full_name('shaq')

In [ ]:
# Find teams by full name.
teams.find_teams_by_full_name('mav')

In [ ]:
from nba_api.stats.endpoints import commonplayerinfo

jordan_id = 893
player_info = commonplayerinfo.CommonPlayerInfo(player_id = jordan_id, timeout=200)

In [ ]:
player_info.available_seasons.get_json()

In [ ]:
player_info.common_player_info.get_data_frame()

In [ ]:
def return_player_info(id):
    player_info = commonplayerinfo.CommonPlayerInfo(player_id = id, timeout=50)
    player_info_df = player_info.common_player_info.get_data_frame()
    
    ## Replace Empty String w/ None
    player_info_df = player_info_df.replace(r'^\s*$', "NOT_AVAILABLE",regex=True).replace('',"NOT_AVAILABLE")
    
    if player_info_df['HEIGHT'].item() != "NOT_AVAILABLE":
        height = float(player_info_df['HEIGHT'].item().replace("-","."))
    else:
        height = np.nan
    if player_info_df['WEIGHT'].item() != "NOT_AVAILABLE":
        weight = int(player_info_df['WEIGHT'].item())
    else:
        weight = np.nan
    
    data = {
        "DISPLAY_FIRST_LAST": player_info_df['DISPLAY_FIRST_LAST'].item(),
        "PERSON_ID": player_info_df['PERSON_ID'].item(),
        "BIRTHDATE": datetime.strptime(player_info_df['BIRTHDATE'].item().split('T')[0],"%Y-%m-%d"),
        "HEIGHT": height,
        "WEIGHT": weight,
        "SCHOOL": player_info_df['SCHOOL'].item(),
        "POSITION": player_info_df['POSITION'].item(),
        "JERSEY": player_info_df['JERSEY'].item() 
    }
    return data

In [ ]:
return_player_info(893)

In [ ]:
player_name = "Michael Jordan"
return_player_info(players.find_players_by_full_name(player_name)[0]['id'])

In [ ]:
player_name = "Shaquille O'Neal"
return_player_info(players.find_players_by_full_name(player_name)[0]['id'])

In [ ]:
player_name = "Larry Bird"
return_player_info(players.find_players_by_full_name(player_name)[0]['id'])

In [ ]:
player_name = "Magic Johnson"
return_player_info(players.find_players_by_full_name(player_name)[0]['id'])

In [ ]:
return_player_info(1626147)

## List Players

In [ ]:
from nba_api.stats.static import players

In [ ]:
active = players.get_active_players()

In [ ]:
inactive = players.get_inactive_players()

In [ ]:
active_ids = [x['id'] for x in active]
print(len(active_ids))

In [ ]:
inactive_ids = [x['id'] for x in inactive]
print(len(inactive_ids))

In [ ]:
## Search for MJ in Active, Inactive players
893 in active_ids, 893 in inactive_ids

In [ ]:
201571
1629028

## Get Carrier info

In [ ]:
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import playoffpicture
from nba_api.stats.endpoints import  playerawards
from nba_api.stats.endpoints import  playerprofilev2

In [ ]:
def return_career_info(id):
    career = playercareerstats.PlayerCareerStats(player_id = id, timeout=50)
    career_df_detailed = career.get_data_frames()[0]
    career_df_summary = career.get_data_frames()[1]

    awards = awards = playerawards.PlayerAwards(id)
    
    data = {
        "GP":  career_df_summary['GP'].item(),
        "PTS":  career_df_summary['PTS'].item(),
        "FG3M": career_df_summary['FG3M'].item(),
        "REB": career_df_summary['REB'].item(),
        "OREB": career_df_summary['OREB'].item(),
        "DREB": career_df_summary['DREB'].item(),
        "AST": career_df_summary['AST'].item(),
        "STL": career_df_summary['STL'].item(),
        "BLK": career_df_summary['BLK'].item(),
        "TOV": career_df_summary['TOV'].item(),
        "PF": career_df_summary['PF'].item(),      

        "TEAMS_PLAYED": list(career_df_detailed['TEAM_ABBREVIATION'].unique()),
        "AWARDS":  list(awards.get_data_frames()[0]['DESCRIPTION'].unique())
    } 
    return data

In [ ]:
### Michael Jordan 893
# Magic Johnson 77142
# Larry Bird 1449
return_career_info(1449)

## DataFrame - Get all Players

In [ ]:
col_info = ['BIRTHDATE', 'DISPLAY_FIRST_LAST','HEIGHT', 'WEIGHT', 'JERSEY', 'POSITION', 'SCHOOL']
col_career = ['GP', 'PTS', 'FG3M', 'REB', 'OREB', 'DREB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'TEAMS_PLAYED', 'AWARDS']

In [ ]:
all_player_df = pd.DataFrame(columns= col_info + col_career )
all_player_df

In [ ]:
print(len(active_ids))
print(len(inactive_ids))
print(len(active_ids+inactive_ids))

In [ ]:
## Adding Active Ids
for i in tqdm_notebook(active_ids+inactive_ids):
    try:
        player_info = return_player_info(i)
        player_career = return_career_info(i)
        entry = player_info.update(player_career)
        all_player_df.append(entry, ignore_index = True)
    except:
        print(i)

In [ ]:
all_player_df

In [ ]:
import pickle

with open("all_player_df.pkl", "wb") as f:
    pickle.dump((all_player_df, f))

## Sources
1. https://towardsdatascience.com/categorical-embedding-and-transfer-learning-dd3c4af6345d
2. https://towardsdatascience.com/link-prediction-with-neo4j-part-2-predicting-co-authors-using-scikit-learn-78b42356b44c  
3. https://medium.com/@davidheffernan_99410/an-introduction-to-using-categorical-embeddings-ee686ed7e7f9  
4. https://medium.com/@george.drakos62/decoded-entity-embeddings-of-categorical-variables-in-neural-networks-1d2468311635  

## Implementations
1. https://www.kaggle.com/keremt/pytorch-entity-embeddings
2. https://pypi.org/project/entity-embeddings-categorical/

## NBA API
1. https://pypi.org/project/nba-api/

## UMAP
1. https://umap-learn.readthedocs.io/